In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

# Chain of Thought Prompting

Beim Chain of Thought (CoT) Promting wird das LLM dazu ermutigt seinen Entscheidungsprozess zu erklären. Dabei wird der Prompt um ein einzelnes oder wenige Beispiele erweitert, die den Entscheidungsprozess abbilden. Somit enthält auch die Antwort des LLMs diesen Prozess, was zu genaueren Ergebnissen führt.

In [2]:
question = (
                "It takes Dave the developer 1.5 hours to fix 1 bug. "
                "After fixing 2 bugs he must rest for 0.5 hours and drink coffee. "
                "How many bugs can he fix in 8.5 hours?"
            )

# CORRECT ANSWER: 5

## Raw Prompt

Entspricht der direkten Eingabe der Aufgabenstellung in ChatGPT (GPT-3.5)

In [3]:
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": question
        }
    ],
    temperature=0
)
print(response["choices"][0]["message"]["content"])

In 8.5 hours, Dave can work for 8.5 - 0.5 = <<8.5-0.5=8>>8 hours.
In 8 hours, Dave can fix 8 / 1.5 = <<8/1.5=5.33>>5.33 bugs.
Since Dave cannot fix a fraction of a bug, he can fix a maximum of 5 bugs. Answer: \boxed{5}.


## One/few Shot standard Prompt (ohne CoT)

- Prompt wird um ein oder mehrere Beispiele ergänzt
- Ähnlich wie Nachtrainieren (fine tuning)
- Zählt als zusätzliche Tokens (Tokenlimit, Kosten)

In [4]:
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        ### Examples
        {
            "role": "system",
            "name": "example_user",
            "content": """It takes Amy 5 minutes to climb and sliding the water slide down. After she slid 4 times, she must rest for 2 minutes. The water slide closes in 50 minutes. How many times can she slide before it closes?"""
        },
        {
            "role": "system",
            "name": "example_assistant",
            "content": """Amy can slide 9 times in the 50 minutes.
            """
        },
        ### Actual question
        {
            "role": "user",
            "content": question
        }
    ],
    temperature=0
)
print(response["choices"][0]["message"]["content"])

Dave can fix 4 bugs in 8.5 hours.


## CoT-Prompting

- Ebenfalls "few shot examples"
- Im Beispiel wird der Denk-/Entscheidungsprozess, nicht nur das erwartete Ergebnis

In [5]:
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        ### Examples
        {
            "role": "system",
            "name": "example_user",
            "content": """It takes Amy 5 minutes to climb and sliding the water slide down. After she slid 4 times, she must rest for 2 minutes. The water slide closes in 50 minutes. How many times can she slide before it closes?"""
        },
        {
            "role": "system",
            "name": "example_assistant",
            "content": """
            Let's break down the scenario into steps and create a plan to solve it:
            
            1. Amy takes 5 minutes for each slide.
            2. After 2 slides (which takes 10 minutes), Amy takes a 2-minute break.
            3. Amy has 50 minutes to take slides.
            
            We can group the time taken to slide 4 times and take a break as a single "slide-block" which lasts for 22 minutes (20 minutes for sliding + 2 minutes for a break).
            Now, we can find the number of complete slide-blocks Amy can have within the 50 minutes: 
            Number of slide-blocks is: 50//22=2
            
            Now, we calculate the total time taken for these 2 slide-blocks: 
            Total time for 2 slide-blocks is: 2x22=44 minutes
            
            Now, the remaining time Amy has is: 50-44=6 minutes
            In this remaining time, Amy can slide one more time as it takes 5 minutes to slide.
            
            Now, the total number of slides Amy can make is the sum of the slides during the slide-blocks and the additional slide in the remaining time:
            Total slides = (2x4 slides per slide-block) + 1 = 8 + 1 = 9
            So, Amy can slide 9 times in the 50 minutes.
            """
        },
        ### Actual question
        {
            "role": "user",
            "content": question
        }
    ],
    temperature=0
)
print(response["choices"][0]["message"]["content"])

Let's break down the scenario into steps and create a plan to solve it:

1. Dave takes 1.5 hours to fix 1 bug.
2. After fixing 2 bugs (which takes 3 hours), Dave takes a 0.5-hour break.
3. Dave has 8.5 hours to fix bugs.

We can group the time taken to fix 2 bugs and take a break as a single "bug-block" which lasts for 3.5 hours (3 hours for fixing bugs + 0.5 hours for a break).
Now, we can find the number of complete bug-blocks Dave can have within the 8.5 hours:
Number of bug-blocks is: 8.5 // 3.5 = 2

Now, we calculate the total time taken for these 2 bug-blocks:
Total time for 2 bug-blocks is: 2 x 3.5 = 7 hours

Now, the remaining time Dave has is: 8.5 - 7 = 1.5 hours
In this remaining time, Dave can fix one more bug as it takes 1.5 hours to fix.

Now, the total number of bugs Dave can fix is the sum of the bugs fixed during the bug-blocks and the additional bug in the remaining time:
Total bugs = (2 x 2 bugs per bug-block) + 1 = 4 + 1 = 5

So, Dave can fix 5 bugs in 8.5 hours.


# Zero Shot Chain of Thought

Das Zero Shot Chain of Thought (Zero-shot-CoT) Prompting ist eine Fortsetzung des CoT Prompting, wobei der eigentliche Prompt nur minimal erweitert wird. Durch das Anhängen der Worte **"Let's think step by step."** an das Ende einer Frage, sind LLMs in der Lage eine Gedankenkette zu erzeugen, aus der sie wiederum genauere Antworten ableiten können.

In [6]:
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": question + " Let's think step by step."
        }
    ],
    temperature=0
)
print(response["choices"][0]["message"]["content"])

Step 1: Calculate the time it takes Dave to fix 2 bugs.
Since it takes Dave 1.5 hours to fix 1 bug, it will take him 1.5 * 2 = <<1.5*2=3>>3 hours to fix 2 bugs.

Step 2: Calculate the total time Dave needs to fix 2 bugs and rest.
After fixing 2 bugs, Dave needs to rest for 0.5 hours, so the total time needed is 3 + 0.5 = <<3+0.5=3.5>>3.5 hours.

Step 3: Calculate the number of times Dave can complete the cycle of fixing 2 bugs and resting in 8.5 hours.
Since each cycle takes 3.5 hours, Dave can complete 8.5 / 3.5 = <<8.5/3.5=2.4285714285714284>>2.43 cycles.

Step 4: Calculate the number of bugs Dave can fix in 8.5 hours.
Since each cycle consists of fixing 2 bugs, Dave can fix 2.43 * 2 = <<2.43*2=4.86>>4.86 bugs.

Step 5: Round down the number of bugs Dave can fix to the nearest whole number.
Since Dave cannot fix a fraction of a bug, he can fix a maximum of 4 bugs in 8.5 hours.


# GPT-4

In [7]:
response = openai.ChatCompletion.create(
    model='gpt-4',
    messages=[
        {
            "role": "user",
            "content": question
        }
    ],
    temperature=0
)
print(response["choices"][0]["message"]["content"])

Dave spends 1.5 hours fixing each bug, so for 2 bugs he spends 1.5 * 2 = <<1.5*2=3>>3 hours.
After fixing 2 bugs, he needs to rest for 0.5 hours, so the total time to fix 2 bugs and rest is 3 + 0.5 = <<3+0.5=3.5>>3.5 hours.
In 8.5 hours, Dave can complete 8.5 / 3.5 = <<8.5/3.5=2.4285714285714284>>2.4285714285714284 cycles of fixing 2 bugs and resting.
Since Dave can't complete a fraction of a cycle, he can complete 2 full cycles, which means he can fix 2 * 2 = <<2*2=4>>4 bugs.
After completing 2 cycles, Dave has spent 2 * 3.5 = <<2*3.5=7>>7 hours.
This leaves 8.5 - 7 = <<8.5-7=1.5>>1.5 hours.
In the remaining time, Dave can fix 1.5 / 1.5 = <<1.5/1.5=1>>1 more bug.
So in total, Dave can fix 4 + 1 = <<4+1=5>>5 bugs in 8.5 hours. Answer: \boxed{5}.
